# RL?

Reinforcement Learning, the third paradigm of machine learning (just because it has word 'learning' inside).
No, just kidding. Learn from rewards.

Good reference: [Reinforcement Learning: An Introduction (2nd edition, draft)](https://webdocs.cs.ualberta.ca/~sutton/book/the-book-2nd.html)

## History

Comes from different subject and study. Control theory, etc...

### Multi-Armed Bandit Problem

stateless RL. Main problem, how to balance exploration/exploitation

<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>
<img src=http://www.winneratslots.com/wp-content/uploads/2014/07/One-Armed-Bandit.jpg style="width: 91px" align='left'/>

In [ ]:
import time
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In order to evaluate the algorithm, we create 2000 experiments with different setting of reward for 10 armed bandit, initialized randomly.

In [ ]:
n_testbed = 2000
n_k = 10
qstar = np.zeros((n_testbed, n_k))

np.random.seed(0)
for t in xrange(n_testbed):
    for k in xrange(n_k):
        qstar[t, k] = np.random.normal(0, 1)

In [ ]:
sample_size = 10000

plt.figure(figsize=(20, 5))
samples = np.zeros((n_k, sample_size))
for i in xrange(n_k):
    samples[i, :] = np.random.normal(qstar[0, i],1, size=sample_size)
    
df = pd.DataFrame(samples.transpose())
sns.violinplot(data=df)

#### Epsilon Greedy
The most basic solution is epsilon greedy algorithm, where at each timesteps there is small probability that we take random action

In [ ]:
def egreedy_bandit(qstar, epsilon):
    """Implement epsilon greedy bandit"""
    
    k = len(qstar)
    Q = np.zeros(k)
    N = np.zeros(k)
    rewards = np.zeros(1000)
    for t in xrange(1000):
        
        if np.random.uniform() > epsilon:
            a = Q.argmax()
        else:
            a = np.random.randint(k)
        
        r = np.random.normal(qstar[a], 1)
        N[a] += 1
        Q[a] = Q[a] + (r - Q[a])/N[a]
        
        rewards[t] = r
    
    return rewards

Simulate with different epsilon parameter

In [ ]:
mean_rewards1 = np.zeros(1000)
mean_rewards2 = np.zeros(1000)
mean_rewards3 = np.zeros(1000)
for i in xrange(n_testbed):
    
    mean_rewards1 += egreedy_bandit(qstar[i, :], 0)
    mean_rewards2 += egreedy_bandit(qstar[i, :], 0.1)
    mean_rewards3 += egreedy_bandit(qstar[i, :], 0.01)
    
mean_rewards1 = mean_rewards1 / n_testbed
mean_rewards2 = mean_rewards2 / n_testbed
mean_rewards3 = mean_rewards3 / n_testbed

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(mean_rewards1, label='e=0')
plt.plot(mean_rewards2, label='e=0.1')
plt.plot(mean_rewards3, label='e=0.01')
plt.ylim((0, 1.5))
plt.ylabel('average reward')
plt.xlabel('timestep')
plt.legend()

#### Upper Confidence Bound
In naive epsilon greedy, the action is picked randomly. However, using UCB we can pick the action based on confidence.
The more the action has been picked, and if it returns cumulative lower rewards, then it will less likely to be picked in the future

In [ ]:
def ucb_bandit(qstar, c):
    """Upper confidence bound"""
    
    k = len(qstar)
    Q = np.zeros(k)
    N = np.ones(k)
    rewards = np.zeros(1000)
    for t in xrange(1000):
        
        At = np.array([x + c*(math.sqrt(math.log(t+1)/N[a])) for (a, x) in enumerate(Q)])
        a = At.argmax()
                
        r = np.random.normal(qstar[a], 1)
        N[a] += 1
        Q[a] = Q[a] + (r - Q[a])/N[a]
        
        rewards[t] = r
        
    return rewards

In [ ]:
mean_rewards1 = np.zeros(1000)
mean_rewards2 = np.zeros(1000)
for i in xrange(n_testbed):
    mean_rewards1 += egreedy_bandit(qstar[i, :], 0.1)
    mean_rewards2 += ucb_bandit(qstar[i, :], 2)

mean_rewards1 = mean_rewards1/n_testbed
mean_rewards2 = mean_rewards2/n_testbed

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(mean_rewards1, label='e=0.1')
plt.plot(mean_rewards2, label='c=2')
plt.ylim((0.2, 1.6))
plt.legend()

#### Contextual multi-armed bandits

Special case of multi-armed bandits where at each timestamp we are given context for the next. One example, maybe the bandits reward get shuffled.

No algorithm given in the books. WTF?

### Markov Decision Problem

<img src='http://i.imgur.com/c4YxfBs.png' style='width: 500px'/>

### Agent Environment

<img src='http://i.imgur.com/voT1qCJ.png' style='width: 500px'>

To implement above system, the environment at least have to have transition function and reward function.
The agent needs to have value function and policy function

### Dynamic Programming

Refer to the solution that have exact state.

Download images here: https://www.dropbox.com/s/egjn29qj87drdwo/pictures.zip?dl=0

In [ ]:
steps = []
for i in xrange(16):
    img = plt.imread('rl/step'+str(i+1)+'.png')
    steps += [img]

In [ ]:
plt.figure(figsize=(30, 5))
for i in xrange(16):
    plt.subplot(1, 16, i+1)
    plt.imshow(steps[i])
    plt.xticks([16+i], [i])
    plt.yticks([0], [''])


Toy example: robot walking
It has 16 possible states depends on the position of the legs and 4 possible actions (move right leg up or down, move right leg forward or backward, move left leg up or down, move left leg forward or backward)



In [ ]:
class RobotWalkingEnvironment(object):
    
    def __init__(self, state):
        
        self.state = state
        self.trans = ((1, 3, 4, 12),    #0
                      (0, 2, 5, 13),    #1
                    (3,1,6,14),
                    (2,0,7,15),
                    (5,7,0,8),
                    (4,6,1,9),
                    (7,5,2,10),
                    (6,4,3,11),
                    (9,11,12,4),
                    (8,10,13,5),
                    (11,9,14,6),
                    (10,8,15,7),
                    (13,15,8,0),
                    (12,14,9,1),
                    (15,13,10,2),
                    (14,12,11,3)
                    )
        
        r = np.zeros((16, 4))
        r[15][1] = 1 #Moving forward by move the leg backward
        r[0][1] = -2 #punish moving backward which is move the leg forward without lift them
        r[0][3] = -2
        r[3][3] = -2
        r[12][1] = -2
        r[2][3] = -2
        r[8][1] = -2
        r[1][3] = -2
        r[4][1] = -2
        r[1][2] = -10 # Lift one leg when other already lifted
        r[2][2] = -10
        r[4][0] = -10
        r[7][0] = -10
        r[8][0] = -10
        r[11][0] = -10
        r[13][2] = -10
        r[14][2] = -10
        self.r = r
        
        
    def go(self, a):
        """Go to the next state given current action"""
        reward = self.r[self.state][a]
        self.state = self.trans[self.state][a]
        
        return reward, self.state
    
    def get_state(self):
        return self.state
        
class QAgent():
    
    def __init__(self):
        
        self.Q = np.random.random((16, 4))
        self.eta = 0.5                   # learning rate
        self.gamma = 0.8                 # discount factor
        self.epsilon = 0.1
    
    def get_action(self, state, train=False):
        if train and np.random.uniform() < self.epsilon:
            return int(np.random.uniform()*4)
        return np.argmax(self.Q[state, :])
    
    def update(self, cur_state, action, new_state, reward):
        self.Q[cur_state, action] += self.eta*(reward + self.gamma*max(self.Q[new_state, :]) - self.Q[cur_state, action])

In [ ]:
%matplotlib notebook

### Policy Iteration

Solved using Bellman Equations


In [ ]:
env = RobotWalkingEnvironment(0)

# The state value function
V = np.zeros(16)
gamma = 0.8

# The policy function which map state to action, randomly given value at the beginning
pi = np.array([int(np.random.random()*4) for i in range(16)])

for p in range(100):
    for s in range(len(pi)):
        pi[s] = np.argmax([env.r[s][a] + gamma*V[env.trans[s][a]] for a in xrange(4)])

        for s in range(len(V)):
            a = pi[s]
            V[s] = env.r[s][a] + gamma*V[env.trans[s][a]]

In [ ]:
env = RobotWalkingEnvironment(0)
state = 0
sequence = [0]
for i in xrange(100):
    nextState = env.trans[state][pi[state]]
    sequence = sequence + [nextState]
    
    plt.imshow(steps[state])
    plt.axis('off')
    plt.gcf().canvas.draw()
    time.sleep(0.5)
    
    state = nextState

### Q Learning

An instance of Temporal Difference Learing in case where we do not have access to the model

In [ ]:
agent = QAgent()

for i in xrange(1):
    state = 0
    env = RobotWalkingEnvironment(state)
    for t in xrange(1000):
        a = agent.get_action(state, train=True)

        reward, new_state = env.go(a)
        agent.update(state, a, new_state, reward)
        
        state = new_state

In [ ]:
# Simulate optimal strategy

states = [0]
state = 0
count = 0
while count <= 100:
    
    a = agent.get_action(state)
    reward, new_state = env.go(a)
    state = new_state
    
    states += [state]    
    count += 1
    
    plt.imshow(steps[state])
    plt.axis('off')
    plt.gcf().canvas.draw()
    time.sleep(0.5)

## OpenAI gym

It is a platform to compare multiple reinforcement learning algorithm. Providing diverse sets of problem to be solved.

In [ ]:
!pip install gym

In [ ]:
import gym
env = gym.make('CartPole-v0')
env.reset()
for t in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    print observation
    if done:
        print 'done'
        break

In [ ]:
env.close()

In [ ]:
print env.action_space
print env.observation_space

print env.observation_space.high
print env.observation_space.low

In [ ]:
env = gym.make('CartPole-v0')
for i_episode in xrange(10):
    observation = env.reset()
    for t in xrange(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print 'Episode {} finished after {} timesteps'.format(i_episode+1, t+1)
            break

In [ ]:
env.close()

In [ ]:
from gym import wrappers
env = gym.make('CartPole-v0')
env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

for i_episode in xrange(10):
    observation = env.reset()
    for t in xrange(100):
        env.render()
        action = env.action_space.sample()
        _, _, done, _ = env.step(action)
        if done:
            print 'done'
            break

In [ ]:
env.close()

In [ ]:
gym.upload('/tmp/cartpole-experiment-1', api_key='sk_RIprDTFeQoWlT0enGNPxg')

In [ ]:
# Exercise, implements Q-learning to solve CartPole problem!